In [ ]:
import pandas as pd
import numpy as np

data = {
    'Truth': [
        "String1_1", "String2_1|String2_2", "String1_1|String1_2|String1_3", "String1_1|String1_2", "String2_1|String2_2",
        "String1_1|String1_2", "String2_1|String2_2|String2_3", "String2_1|String2_2", "String1_1|String1_2|String1_3", "String2_1|String2_2",
        "String1_1|String1_2", "String2_1|String2_2"
    ] * 16,
    'Prediction': [
        "String1_1|String1_2|String1_3", "String2_1|String2_2", "String1_1|String1_2", "String2_1|String2_2", "String2_1|String2_2",
        "String1_1|String1_2", "String1_1|String1_2|String1_3", "String2_1|String2_2", "String1_1|String1_2", "String2_1|String2_2",
        "String1_1|String1_2", "String1_1|String1_2|String1_3"
    ] * 16,
    'Subtypes': ['A', 'A', 'A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'B', 'B'] * 8 + ['C', 'C', 'C', 'C', 'C', 'C', 'D', 'D', 'D', 'D', 'D', 'D'] * 8,
}
df = pd.DataFrame(data)
df['ID'] = range(1, len(df) + 1)

# Generate NaN values in the Prediction and Truth columns
df['Prediction'] = np.where(df['ID'] % 3 == 0, np.nan, df['Prediction'])
df['Truth'] = np.where(df['ID'] % 5 == 0, np.nan, df['Truth'])

df['Truth'] = df['Truth'].apply(lambda x: x.split('|') if isinstance(x, str) else [])
df['Prediction'] = df['Prediction'].apply(lambda x: x.split('|') if isinstance(x, str) else [])

mlb = MultiLabelBinarizer()
binarized_truth = mlb.fit_transform(df['Truth'])
binarized_prediction = mlb.transform(df['Prediction'])

def calculate_confusion(row):
    if row['Truth'] is None or row['Prediction'] is None:
        return {'TP': set(), 'FP': set(), 'FN': set(), 'TN': set(mlb.classes_)}
    tp = set(row['Truth']).intersection(set(row['Prediction']))
    fp = set(row['Prediction']).difference(set(row['Truth']))
    fn = set(row['Truth']).difference(set(row['Prediction']))
    tn = set(mlb.classes_).difference(set(row['Prediction']))
    
    return {'TP': tp, 'FP': fp, 'FN': fn, 'TN': tn}

subtypes = df['Subtypes'].unique()
subtype_summaries = {}
for subtype in subtypes:
    mask = df['Subtypes'] == subtype
    subtype_data = df[mask]
    subtype_summaries[subtype] = subtype_data

for subtype in subtypes:
    subtype_data = subtype_summaries[subtype]
    confusion_data = subtype_data.apply(calculate_confusion, axis=1)
    
    tp = sum([len(x['TP']) for x in confusion_data])
    fp = sum([len(x['FP']) for x in confusion_data])
    fn = sum([len(x['FN']) for x in confusion_data])
    tn = sum([len(x['TN']) for x in confusion_data])
    
    if (tp + fp) == 0:
        precision = 0
    else:
        precision = tp / (tp + fp)
    if (tp + fn) == 0:
        recall = 0
    else:
        recall = tp / (tp + fn)
    if (precision + recall) == 0:
        f1_score = 0
    else:
        f1_score = 2 * (precision * recall) / (precision + recall)
    
    with pd.ExcelWriter(os.path.join(r'C:\Users\sigar\Documents\Test', f'{subtype}_metrics.xlsx')) as writer:
        pd.DataFrame({'Precision': [precision], 'Recall': [recall], 'F1-score': [f1_score]}).to_excel(writer, sheet_name='accuracy_score', index=False)
        subtype_data[confusion_data.apply(lambda x: bool(x['FN']))].to_excel(writer, sheet_name='FN', index=False)
        subtype_data[confusion_data.apply(lambda x: bool(x['FP']))].to_excel(writer, sheet_name='FP', index=False)
        subtype_data[confusion_data.apply(lambda x: bool(x['TP']))].to_excel(writer, sheet_name='TP', index=False)
        subtype_data[confusion_data.apply(lambda x: bool(x['TN']))].to_excel(writer, sheet_name='TN', index=False)

